## Libraries

In [257]:
import re
import string

import numpy as np
import pandas as pd
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import emoji
import unidecode
import spacy
from nltk.stem import PorterStemmer


pd.set_option('max_colwidth', 500)
stop_words = spacy.lang.en.stop_words.STOP_WORDS

## Read data

In [2]:
train_path = '../datasets/emotion_detection_semeval2017/combined/train.csv'

In [3]:
train = pd.read_csv(train_path)

In [4]:
train.sample(frac=1).head(10)

,id,tweet,emotion,intensity
3650,10894,i live and die for mchanzo honeymoon crashing and burning the second they move in together,anger,0.479
2318,30675,All the fans wanted Man Utd at home in the next round...\nAre you cheering for Northampton or Man Utd right now?\n#lufc,joy,0.292
1808,30165,A joyous first webiversary/web mitzvah to Smithsonian's @WeiPoints!! @brianwolly @jackie_mansky @bethpylieberman @bilbo @mazeltov,joy,0.667
3494,21028,"@PanicAtTheDisco hey, y'all announced it like immediately after I asked. Nice. Thanks y'all",fear,0.250
1533,40676,Do not be discouraged by a slowing sales market. This will test your business model and pinpoint #strengths and #weaknesses.' @Ken_Dunn,sadness,0.271
3675,10919,"Why to have vanity sizes?Now sizes S,XS(evenXXS sometimes) are too big, WTF?! Dear corporate jerks, Lithuania didn't need this. #rant #angry",anger,0.708
2833,20367,@BaileyDemented @hsmitty3 ill kill u if u bully her 😤😤😤,fear,0.583
3512,21046,Not sure that men can handle a woman that's got her crap together. #independent,fear,0.229
2186,30543,lol! no mention of pak PM or even his speech on any international news channel and pakis are rejoicing as if the world stands with them,joy,0.396
1278,40421,"@DxfyingGrxvity - that were rather forlorn, scanning the witches house before resting back on Elphie. 'The Grimmerie is gone.'",sadness,0.458


In [5]:
sample_tweet = "The moment you bring her to meet your best friend and you're nervous af! 😬😆 #nervous #thefriendtest"

## Preprocessing

In [6]:
# Initialize Ekphrasis parsers
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", # Options are 'twitter' or 'english'
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=False).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

spacy_preprocessor = TextPreProcessor(
    omit=['email', 'phone', 'user', 'time', 'url', 'date', 'hashtag'],
    corrector='twitter',
    segmenter='twitter',
    tokenizer=lambda x: x.split(),
    annotate=[]
)

/home/sanjyot/miniconda3/envs/prnn/lib/python3.7/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...


/home/sanjyot/miniconda3/envs/prnn/lib/python3.7/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading twitter - 1grams ...
Reading twitter - 1grams ...
You can't omit/backoff and unpack hashtags!
 unpack_hashtags will be set to False


In [7]:
# Initialize spacy parser
nlp = spacy.load('en_core_web_lg')

In [327]:
x, _, _ = preprocess_raw_tweet(sample_tweet)
x

['The',
 'moment',
 'you',
 'bring',
 'her',
 'to',
 'meet',
 'your',
 'best',
 'friend',
 'and',
 'you',
 'are',
 'nervous',
 'af',
 '!',
 'grimacing',
 'grinning',
 'squinting',
 '<hashtag>',
 'nervous',
 '</hashtag>',
 '<hashtag>',
 'the',
 'friend',
 'test',
 '</hashtag>']

In [331]:
def preprocess_raw_tweet(twt, preprocessor=text_processor, spacy_preprocessor=nlp):
    
    # Demojize
    def demojize(parsed):
        if emoji.emoji_count(' '.join(parsed)) > 0:
            emoji_parse = []
            for tok in parsed:
                if emoji.emoji_count(tok) > 0:
                    emoji_list = emoji.demojize(tok).strip(':').split('_')
                    emoji_list.remove('face')
                    emoji_parse.extend(emoji_list)
                else:
                    emoji_parse.append(tok)
        else:
            emoji_parse = parsed
        return emoji_parse
    
    def clean_accents(parsed):
        return [unidecode.unidecode(x) for x in parsed]
    
    def clean_tweet_for_spacy(twt):
        # Remove emoji, hashtags, user handles and diacritics/accents
        # and return the sentence as a string
        cleaned = re.sub(r'#\w+', ' ', twt)
        emoji_cleaned = ''
        emoji_count = 0
        for char in cleaned:
            if char in emoji.UNICODE_EMOJI:
                emoji_count += 1
                continue
            emoji_cleaned += char

        emoji_cleaned = re.sub(r'\s\s+', ' ', emoji_cleaned)
        return emoji_cleaned, emoji_count
    
    def extract_hashtag(ekphrasis_parse):
        hashtag_tokens = []
        append_flag = False
        for i, tok in enumerate(ekphrasis_parse):
            if tok == '<hashtag>':
                append_flag = True
            if tok == '</hashtag>':
                append_flag = False

            if append_flag and ekphrasis_parse[i+1] != '</hashtag>':
                hashtag_tokens.append(ekphrasis_parse[i+1])
        return hashtag_tokens
    
    preprocessed_tweet = clean_accents(demojize(preprocessor.pre_process_doc(twt)))
    hashtag_list = extract_hashtag(preprocessed_tweet)
    spacy_raw_text, emoji_count = clean_tweet_for_spacy(twt)
    text_and_hashtag = spacy_raw_text + ' ' + ' '.join(hashtag_list)
    spacy_preprocessed_doc = nlp(text_and_hashtag)
    return preprocessed_tweet, spacy_preprocessed_doc, emoji_count

In [332]:
print(sample_tweet)
print(preprocess_raw_tweet(sample_tweet))

The moment you bring her to meet your best friend and you're nervous af! 😬😆 #nervous #thefriendtest
(['The', 'moment', 'you', 'bring', 'her', 'to', 'meet', 'your', 'best', 'friend', 'and', 'you', 'are', 'nervous', 'af', '!', 'grimacing', 'grinning', 'squinting', '<hashtag>', 'nervous', '</hashtag>', '<hashtag>', 'the', 'friend', 'test', '</hashtag>'], The moment you bring her to meet your best friend and you're nervous af!  nervous the friend test, 2)


## Feature engineering

In [252]:
ekphrasis_parse, spacy_obj, emoji_count = preprocess_raw_tweet(sample_tweet)

In [253]:
spacy_obj

The moment you bring her to meet your best friend and you're nervous af! nervous thefriendtest

In [254]:
# No. of tokens
def get_no_of_tokens(spacy_obj):
    return len(spacy_obj)

# Average token length
def avg_token_length(spacy_obj):
    len_list = [len(x) for x in spacy_obj]
    return float(np.mean(np.array(len_list)))
    
# Upper case token ratio
def upper_case_tokens(spacy_obj):
    return len([x for x in spacy_obj if x.is_upper])/len(spacy_obj)

# Title case token ratio
def title_case_tokens(spacy_obj):
    return len([x for x in spacy_obj if x.is_title])/len(spacy_obj)

# Get exclamation mark counts
def exclamation_mark_count(spacy_obj):
    return len([x for x in spacy_obj if x.text == '!'])

# Get question mark counts
def question_mark_count(spacy_obj):
    return len([x for x in spacy_obj if x.text == '?'])

# Get quote mark counts
def quote_mark_count(spacy_obj):
    return len([x for x in spacy_obj if x.text == '"' or x.text == "'"])

In [255]:
# Extracting 'n' most common tokens for each emotion
def get_n_most_valuable_tokens(train):
    
    n_valuable = (
        train
        [['tweet', 'emotion', 'intensity']]
        .assign(intensity = lambda x: 2*x['intensity'] - 1)
        .assign(tweet = lambda x: x['tweet'].str.split()) # Split on whitespace
        .explode('tweet')
        .assign(tweet = lambda x: x['tweet'].str.replace('[^\w\s]','').str.strip().str.lower()) # Remove punctuation, strip, lower
        .loc[lambda x: ~x['tweet'].isin(stop_words)]  # Remove stop words
        .groupby(by=['tweet', 'emotion'])
        .agg({'intensity': 'mean', 'emotion': 'count'})
        .rename(columns={'emotion': 'word_count', 'intensity': 'avg_intensity'})
        .reset_index()
        .assign(abs_intensity = lambda x: np.abs(x['avg_intensity']))
        .sort_values(by=['abs_intensity'], ascending=False)
        .loc[lambda x: x['word_count'] > 5]
        .loc[lambda x: x['abs_intensity'] > 1e-3]
        .loc[lambda x: x['tweet'] != '']
        
    )
    return n_valuable

def get_n_most_valuable_token_score(spacy_obj, emotion, n_valuable_data):
    score = 0
    for tok in spacy_obj:
        try:
            score += n_valuable_data.loc[(n_valuable_data['tweet'] == tok.text) & (n_valuable_data['emotion'] == emotion), 'avg_intensity'].iloc[0]
        except IndexError:
            pass
    return score
    
n_valuable_data = get_n_most_valuable_tokens(train)

get_n_most_valuable_token_score(nlp(sample_tweet), 'fear', n_valuable_data)

1.0325666666666669

In [262]:
# Presence of synonyms and antonyms of the emotion labels
def syn_ant_score(spacy_obj):
    ps= PorterStemmer()
    
    joy_syn = ["joy","happy","content","statisfy","satisfactory","amusement","bliss","charm","cheer","comfort","delight","elation","glee","humor","pride","satisfaction","wonder","alleviatin","animation","delectation","diversion","ecstasy","exultation","exulting","felicity","festivity","frolic","fruition","gaiety","gem","gladness","gratification","hilarity","indulgence","jewel","jubilance","liveliness","luxury","merriment","mirth","prize","rapture","ravishment","refreshment","rejoicing","revelry","solace","sport","transport","treasure","treat","good humor","pride and joy","regalement"]
    joy_ant = ["sad","depression","melancholy","misery","sadness","seriousness","sorrow","unhappiness","discouragement","dislike","mourning","vocation","work","woe"]
    sad_syn= ["sad","bitter","dismal","heartbroken","melancholy","mournful","pessimistic","somber","sorrowful","sorry","wistful","bereaved","blue","cheerless","dejected","despairing","despondent","disconsolate","distressed","doleful","down","down in dumps","down in mouth","downcast","forlorn","gloomy","glum","grief-stricken","grieved","heartsick","heavyhearted","hurting","in doldrums","in grief","in the dumps","languishing","low","low-spirited","lugubrious","morbid","morose","out of sorts","pensive","sick at heart","troubled","weeping","woebegone"]
    sad_ant = ["joy","satisfy","delightful","glad","good","happy","hopeful","joyful","nice","pleasant","fortunate","great","lucky"]
    anger_syn = ["acrimony","animosity","annoyance","antagonism","displeasure","enmity","exasperation","fury","hatred","impatience","indignation","ire","irritation","outrage","passion","rage","resentment","temper","violence","chagrin","choler","conniption","dander","disapprobation","distemper","gall","huff","infuriation","irascibility","irritability","miff","peevishness","petulance","pique","rankling","soreness","stew","storm","tantrum","tiff","umbrage","vexation","blow up","cat fit","hissy fit","ill humor","ill temper","mad","slow burn"]
    anger_ant = ["calm","calmness","cheer","comfort","delight","ease","glee","good will","happiness","joy","kindness","liking","love","peace","pleasure","agreeability","contentment","enjoyment","good nature","pleasantness"]
    fear_syn= ["angst","anxiety","concern","despair","dismay","doubt","dread","horror","jitters","panic","scare","suspicion","terror","unease","uneasiness","worry","abhorrence","agitation","aversion","awe","consternation","cowardice","creeps","discomposure","disquietude","distress","faintheartedness","foreboding","fright","funk","misgiving","nightmare","phobia","presentiment","qualm","reverence","revulsion","timidity","trembling","tremor","trepidation","bête noire","chickenheartedness","cold feet","cold sweat","recreancy"]
    fear_ant = ["assurance","calmness","cheer","confidence","contentment","ease","encouragement","faith","happiness","joy","trust","calm","comfort","like","liking","love","bravery","courage","fearlessness","heroism","unconcern"]

    joy_syn = [ps.stem(x) for x in joy_syn]
    joy_ant = [ps.stem(x) for x in joy_ant]
    sad_syn = [ps.stem(x) for x in sad_syn]
    sad_ant = [ps.stem(x) for x in sad_ant]
    anger_syn = [ps.stem(x) for x in anger_syn]
    anger_ant = [ps.stem(x) for x in anger_ant]
    fear_syn = [ps.stem(x) for x in fear_syn]
    fear_ant = [ps.stem(x) for x in fear_ant]

    joy_score = 0
    sad_score = 0
    fear_score = 0
    anger_score = 0
    
    for tok in spacy_obj:
        st_tok = ps.stem(tok.text.lower())
        
        if st_tok in joy_syn:
            joy_score += 1
        if st_tok in joy_ant:
            joy_score -= 1
        if st_tok in sad_syn:
            sad_score += 1
        if st_tok in sad_ant:
            sad_score -= 1
        if st_tok in fear_syn:
            fear_score += 1
        if st_tok in fear_ant:
            fear_score -= 1
        if st_tok in anger_syn:
            anger_score += 1
        if st_tok in anger_ant:
            anger_score -= 1
    
    return joy_score, sad_score, fear_score, anger_score

In [161]:
# Swear words
def get_swear_word_list():
    with open('../datasets/swear_words.csv', 'r') as f:
        swear_words = f.read().split(',')

    with open('../datasets/swear_words_2.csv', 'r') as f:
        swear_words_2 = f.read().split('\n')
    swear = list(set(swear_words + swear_words_2))
    swear.remove('')
    return swear

swear = get_swear_word_list()

def get_no_of_swear_words(spacy_obj, swear=swear):
    c = 0
    lowercase_tweet = [x.text.lower() for x in spacy_obj]
    for t in lowercase_tweet:
        if t in swear:
            c += 1
    return c

In [190]:
# Embedding features

def get_embedding_features(spacy_obj):
    joy_vec = nlp('joy')[0]
    sad_vec = nlp('sad')[0]
    anger_vec = nlp('anger')[0]
    fear_vec = nlp('fear')[0]
    
    embed_words = [x for x in spacy_obj if x not in stop_words and x.is_alpha]
    
    avg_joy_similarity = float(np.mean([x.similarity(joy_vec) for x in embed_words]))
    avg_sad_similarity = float(np.mean([x.similarity(sad_vec) for x in embed_words]))
    avg_anger_similarity = float(np.mean([x.similarity(anger_vec) for x in embed_words]))
    avg_fear_similarity = float(np.mean([x.similarity(fear_vec) for x in embed_words]))
    
    max_joy_similarity = float(np.max([x.similarity(joy_vec) for x in embed_words]))
    max_sad_similarity = float(np.max([x.similarity(sad_vec) for x in embed_words]))
    max_anger_similarity = float(np.max([x.similarity(anger_vec) for x in embed_words]))
    max_fear_similarity = float(np.max([x.similarity(fear_vec) for x in embed_words]))
    
    return [avg_joy_similarity, avg_sad_similarity, avg_anger_similarity, avg_fear_similarity,
            max_joy_similarity, max_sad_similarity, max_anger_similarity, max_fear_similarity]

In [191]:
get_embedding_features(spacy_obj)

[0.3634510636329651,
 0.3534475862979889,
 0.3017246723175049,
 0.38022226095199585,
 0.570914626121521,
 0.5941110253334045,
 0.434806227684021,
 0.5261985659599304]

In [213]:
spacy_obj

The moment you bring her to meet your best friend and you're nervous af!

In [314]:
# Affect intensity score
affect_score_filepath = '../datasets/lexicon_sentiments/NRC-Sentiment-Emotion-Lexicons/NRC-Affect-Intensity-Lexicon/NRC-AffectIntensity-Lexicon.txt'
score_df = pd.read_csv(affect_score_filepath, sep='\t')

def get_affect_intensity_score(spacy_obj, emotion, score_df=score_df):
    score = 0
    for tok in spacy_obj:
        try:
            score += score_df.loc[(score_df['term'] == tok.text.lower()) & (score_df['AffectDimension'] == emotion), 'score'].iloc[0]
        except IndexError:
            pass
    return score

In [238]:
# Get lexicon senselevel data

senselevel_data_path = '../datasets/lexicon_sentiments/NRC-Sentiment-Emotion-Lexicons/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-Senselevel-v0.92.txt'
senselevel_data_score_df = (
    pd.read_csv(senselevel_data_path, sep='\t', names=['word', 'emotion', 'score'])
    .assign(word = lambda x: x['word'].str.split('--').str[0])
    .drop_duplicates()
    .loc[lambda x: x['score'] > 0]
    .loc[lambda x: x['emotion'].isin(['fear', 'anger', 'joy', 'sadness'])]      
)

In [272]:
# Lexicon senselevel data score
def get_lexicon_senselevel_data_score(spacy_obj, emotion, score_df=senselevel_data_score_df):
    score = 0
    for tok in spacy_obj:
        try:
            score += score_df.loc[(score_df['word'] == tok.text.lower()) & (score_df['emotion'] == emotion), 'score'].iloc[0]
        except IndexError:
            pass
    return score
    
get_lexicon_senselevel_data_score(spacy_obj, 'fear')

2

## Collect features into single function

In [339]:
def generate_all_features(df, emotion):
    
    edf = df[df['emotion'] == emotion]
    
    def initialize_feat_dict():
        id_cols = ['id', 'intensity']
        syn_features = ['no_of_tokens', 'avg_token_len', 'upper_case_tokens', 'title_case_tokens', 
                    'exclamation_mark_count', 'question_mark_count', 'quote_mark_count', 
                    'emoji_count']
        sem_features = ['n_valuable_tok_score', 'joy_syn_ant_score', 'sad_syn_ant_score', 
                        'fear_syn_ant_score', 'anger_syn_ant_score', 'cuss_count']
        embedding_features = ['avg_joy_sim', 'avg_sad_sim', 'avg_anger_sim', 'avg_fear_sim', 
                              'max_joy_sim', 'max_sad_sim', 'max_anger_sim', 'max_fear_sim']
        affect_data_features = ['joy_affect_score', 'sad_affect_score', 'anger_affect_score', 'fear_affect_score']
        
        senselevel_data_features = ['joy_senselevel_score', 'sad_senselevel_score', 'anger_senselevel_score', 'fear_senselevel_score']
        
        feat = dict()
        for col in id_cols + syn_features + sem_features + embedding_features + affect_data_features + senselevel_data_features:
            feat[col] = list()
            
        return feat
    
    feat = initialize_feat_dict()  # Dictionary to store results
    
    for _, row in edf.iterrows():
        row = row.to_dict()
        tweet_id = row['id']
        tweet = row['tweet']
        intensity = row['intensity']
        
        feat['id'] = tweet_id
        feat['intensity'] = intensity
        
        annotated_tweet, spacy_obj, emoji_count = preprocess_raw_tweet(tweet)
        
        # Syntactic features
        feat['no_of_tokens'].append(get_no_of_tokens(spacy_obj))
        feat['avg_token_len'].append(avg_token_length(spacy_obj))
        feat['upper_case_tokens'].append(upper_case_tokens(spacy_obj))
        feat['title_case_tokens'].append(title_case_tokens(spacy_obj))
        feat['exclamation_mark_count'].append(exclamation_mark_count(spacy_obj))
        feat['question_mark_count'].append(question_mark_count(spacy_obj))
        feat['quote_mark_count'].append(quote_mark_count(spacy_obj))
        feat['emoji_count'].append(emoji_count)
        
        # Semantic features
        feat['n_valuable_tok_score'].append(get_n_most_valuable_token_score(spacy_obj, emotion, n_valuable_data=n_valuable_data))
        joy_syn_ant_score, sad_syn_ant_score, fear_syn_ant_score, anger_syn_ant_score = syn_ant_score(spacy_obj)
        feat['joy_syn_ant_score'].append(joy_syn_ant_score)
        feat['sad_syn_ant_score'].append(sad_syn_ant_score)
        feat['fear_syn_ant_score'].append(fear_syn_ant_score)
        feat['anger_syn_ant_score'].append(anger_syn_ant_score)
        feat['cuss_count'].append(get_no_of_swear_words(spacy_obj))
        
        # Embedding features
        avg_joy_sim, avg_sad_sim, avg_anger_sim, avg_fear_sim, \
        max_joy_sim, max_sad_sim, max_anger_sim, max_fear_sim = get_embedding_features(spacy_obj)
        feat['avg_joy_sim'].append(avg_joy_sim)
        feat['avg_sad_sim'].append(avg_sad_sim)
        feat['avg_anger_sim'].append(avg_anger_sim)
        feat['avg_fear_sim'].append(avg_fear_sim)
        feat['max_joy_sim'].append(max_joy_sim)
        feat['max_sad_sim'].append(max_sad_sim)
        feat['max_anger_sim'].append(max_anger_sim)
        feat['max_fear_sim'].append(max_fear_sim)
        
        # Affect data intensity score
        feat['joy_affect_score'].append(get_affect_intensity_score(spacy_obj, 'joy'))
        feat['sad_affect_score'].append(get_affect_intensity_score(spacy_obj, 'sadness'))
        feat['anger_affect_score'].append(get_affect_intensity_score(spacy_obj, 'anger'))
        feat['fear_affect_score'].append(get_affect_intensity_score(spacy_obj, 'fear'))
        
        # Get lexicon senselevel data score
        feat['joy_senselevel_score'].append(get_lexicon_senselevel_data_score(spacy_obj, 'joy'))
        feat['sad_senselevel_score'].append(get_lexicon_senselevel_data_score(spacy_obj, 'sadness'))
        feat['anger_senselevel_score'].append(get_lexicon_senselevel_data_score(spacy_obj, 'anger'))
        feat['fear_senselevel_score'].append(get_lexicon_senselevel_data_score(spacy_obj, 'fear'))
    return pd.DataFrame(feat)

In [337]:
train.head()

,id,tweet,emotion,intensity
0,10000,How the fu*k! Who the heck! moved my fridge!... should I knock the landlord door. #angry #mad ##,anger,0.938
1,10001,So my Indian Uber driver just called someone the N word. If I wasn't in a moving vehicle I'd have jumped out #disgusted,anger,0.896
2,10002,@DPD_UK I asked for my parcel to be delivered to a pick up store not my address #fuming #poorcustomerservice,anger,0.896
3,10003,so ef whichever butt wipe pulled the fire alarm in davis bc I was sound asleep #pissed #angry #upset #tired #sad #tired #hangry ######,anger,0.896
4,10004,"Don't join @BTCare they put the phone down on you, talk over you and are rude. Taking money out of my acc willynilly! #fuming",anger,0.896


In [340]:
generate_all_features(train.head(3), 'anger')

,id,intensity,no_of_tokens,avg_token_len,upper_case_tokens,title_case_tokens,exclamation_mark_count,question_mark_count,quote_mark_count,emoji_count,...,max_anger_sim,max_fear_sim,joy_affect_score,sad_affect_score,anger_affect_score,fear_affect_score,joy_senselevel_score,sad_senselevel_score,anger_senselevel_score,fear_senselevel_score
0,10002,0.896,24,3.208333,0.041667,0.125000,3,0,0,0,...,0.683652,0.574110,0.000,0.500,1.512,0.547,0,1,2,1
1,10002,0.896,27,3.592593,0.111111,0.259259,0,0,0,0,...,0.465649,0.499185,0.000,0.295,0.000,0.000,0,0,0,0
2,10002,0.896,22,4.045455,0.090909,0.045455,0,0,0,0,...,0.433860,0.532098,0.172,0.000,0.812,0.000,0,0,1,0
